In [35]:
import numpy as np

from performance_metrics import calc_root_mean_squared_error

def make_train_and_test_row_ids_for_n_fold_cv(
        n_examples=0, n_folds=3, random_state=0, test_size=0.2):
    ''' Divide row ids into train and test sets for n-fold cross validation.

    Will *shuffle* the row ids via a pseudorandom number generator before
    dividing into folds.

    Args
    ----
    n_examples : int
        Total number of examples to allocate into train/test sets
    n_folds : int
        Number of folds requested
    random_state : int or numpy RandomState object
        Pseudorandom number generator (or seed) for reproducibility

    Returns
    -------
    train_ids_per_fold : list of 1D np.arrays
        One entry per fold
        Each entry is a 1-dim numpy array of unique integers between 0 to N
    test_ids_per_fold : list of 1D np.arrays
        One entry per fold
        Each entry is a 1-dim numpy array of unique integers between 0 to N

    Guarantees for Return Values
    ----------------------------
    Across all folds, guarantee that no two folds put same object in test set.
    For each fold f, we need to guarantee:
    * The *union* of train_ids_per_fold[f] and test_ids_per_fold[f]
    is equal to [0, 1, ... N-1]
    * The *intersection* of the two is the empty set
    * The total size of train and test ids for any fold is equal to N

    Examples
    --------
    >>> N = 11
    >>> n_folds = 3
    >>> tr_ids_per_fold, te_ids_per_fold = (
    ...     make_train_and_test_row_ids_for_n_fold_cv(N, n_folds))
    >>> len(tr_ids_per_fold)
    3

    # Count of items in training sets
    >>> np.sort([len(tr) for tr in tr_ids_per_fold])
    array([7, 7, 8])

    # Count of items in the test sets
    >>> np.sort([len(te) for te in te_ids_per_fold])
    array([3, 4, 4])

    # Test ids should uniquely cover the interval [0, N)
    >>> np.sort(np.hstack([te_ids_per_fold[f] for f in range(n_folds)]))
    array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

    # Train ids should cover the interval [0, N) TWICE
    >>> np.sort(np.hstack([tr_ids_per_fold[f] for f in range(n_folds)]))
    array([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
            8,  9,  9, 10, 10])
    '''
    if hasattr(random_state, 'rand'):
        # Handle case where provided random_state is a random generator
        # (e.g. has methods rand() and randn())
        random_state = random_state # just remind us we use the passed-in value
    else:
        # Handle case where we pass "seed" for a PRNG as an integer
        random_state = np.random.RandomState(int(random_state))

    # TODO obtain a shuffled order of the n_examples
    indeces = np.arange(0, n_examples, 1)
    random_state.shuffle(indeces)    
    
    # TODO establish the row ids that belong to each fold's
    # train subset and test subset
    
    test_ids_per_fold = np.array_split(indeces, n_folds)
    train_ids_per_fold = [
        np.concatenate(list(filter(
            lambda current_train_fold: not np.array_equal(current_train_fold, current_test_fold), test_ids_per_fold
        ))) for current_test_fold in test_ids_per_fold
    ]
    

    return train_ids_per_fold, test_ids_per_fold

In [36]:
def train_models_and_calc_scores_for_n_fold_cv(
        estimator, x_NF, y_N, n_folds=3, random_state=0):
    ''' Perform n-fold cross validation for a specific sklearn estimator object

    Args
    ----
    estimator : any regressor object with sklearn-like API
        Supports 'fit' and 'predict' methods.
    x_NF : 2D numpy array, shape (n_examples, n_features) = (N, F)
        Input measurements ("features") for all examples of interest.
        Each row is a feature vector for one example.
    y_N : 1D numpy array, shape (n_examples,)
        Output measurements ("responses") for all examples of interest.
        Each row is a scalar response for one example.
    n_folds : int
        Number of folds to divide provided dataset into.
    random_state : int or numpy.RandomState instance
        Allows reproducible random splits.

    Returns
    -------
    train_error_per_fold : 1D numpy array, size n_folds
        One entry per fold
        Entry f gives the error computed for train set for fold f
    test_error_per_fold : 1D numpy array, size n_folds
        One entry per fold
        Entry f gives the error computed for test set for fold f

    Examples
    --------
    # Create simple dataset of N examples where y given x
    # is perfectly explained by a linear regression model
    >>> N = 101
    >>> n_folds = 7
    >>> x_N3 = np.random.RandomState(0).rand(N, 3)
    >>> y_N = np.dot(x_N3, np.asarray([1., -2.0, 3.0])) - 1.3337
    >>> y_N.shape
    (101,)

    >>> import sklearn.linear_model
    >>> my_regr = sklearn.linear_model.LinearRegression()
    >>> tr_K, te_K = train_models_and_calc_scores_for_n_fold_cv(
    ...                 my_regr, x_N3, y_N, n_folds=n_folds, random_state=0)

    # Training error should be indistiguishable from zero
    >>> np.array2string(tr_K, precision=8, suppress_small=True)
    '[0. 0. 0. 0. 0. 0. 0.]'

    # Testing error should be indistinguishable from zero
    >>> np.array2string(te_K, precision=8, suppress_small=True)
    '[0. 0. 0. 0. 0. 0. 0.]'
    '''
    train_error_per_fold = np.zeros(n_folds, dtype=np.float32)
    test_error_per_fold = np.zeros(n_folds, dtype=np.float32)

    # TODO define the folds here by calling your function
    # e.g. ... = make_train_and_test_row_ids_for_n_fold_cv(...)
    train_ids_per_fold, test_ids_per_fold = make_train_and_test_row_ids_for_n_fold_cv(x_NF.shape[0], n_folds=n_folds, random_state=random_state)

    # TODO loop over folds and compute the train and test error
    # for the provided estimator
            
    train_error_per_fold = np.array([])
    test_error_per_fold = np.array([])
    for idx in range(n_folds):
        train_idxs = train_ids_per_fold[idx]
        test_idxs = test_ids_per_fold[idx]
        
        x_curr_train, x_curr_test = x_NF[train_idxs], x_NF[test_idxs]
        y_curr_train, y_curr_test = y_N[train_idxs], y_N[test_idxs]
        estimator.fit(x_curr_train, y_curr_train)
        y_pred_train = estimator.predict(x_curr_train)
        y_pred_test = estimator.predict(x_curr_test)
        
        train_error_per_fold = np.append(
            train_error_per_fold, 
            [calc_root_mean_squared_error(y_curr_train, y_pred_train)]
            )
        test_error_per_fold = np.append(
            test_error_per_fold,
            [calc_root_mean_squared_error(y_curr_test, y_pred_test)]
        )
        
        

    return train_error_per_fold, test_error_per_fold

In [33]:
    # Create simple dataset of N examples where y given x
    # is perfectly explained by a linear regression model
N = 101
n_folds = 7
x_N3 = np.random.RandomState(0).rand(N, 3)
y_N = np.dot(x_N3, np.asarray([1., -2.0, 3.0])) - 1.3337
y_N.shape


(101,)

In [37]:
import sklearn.linear_model
my_regr = sklearn.linear_model.LinearRegression()
tr_K, te_K = train_models_and_calc_scores_for_n_fold_cv(
    my_regr, 
    x_N3, 
    y_N, 
    n_folds=n_folds, 
    random_state=0
)


In [38]:
# Training error should be indistiguishable from zero
np.array2string(tr_K, precision=8, suppress_small=True)
#'[0. 0. 0. 0. 0. 0. 0.]'

'[0. 0. 0. 0. 0. 0. 0.]'

In [39]:
# Testing error should be indistinguishable from zero
np.array2string(te_K, precision=8, suppress_small=True)
#'[0. 0. 0. 0. 0. 0. 0.]'

'[0. 0. 0. 0. 0. 0. 0.]'

## Split Funcn Debug

In [13]:
test_n_examples=10
test_n_folds=3
test_random_state=0
''' Divide row ids into train and test sets for n-fold cross validation.

    Will *shuffle* the row ids via a pseudorandom number generator before
    dividing into folds.

    Args
    ----
    n_examples : int
        Total number of examples to allocate into train/test sets
    n_folds : int
        Number of folds requested
    random_state : int or numpy RandomState object
        Pseudorandom number generator (or seed) for reproducibility

    Returns
    -------
    train_ids_per_fold : list of 1D np.arrays
        One entry per fold
        Each entry is a 1-dim numpy array of unique integers between 0 to N
    test_ids_per_fold : list of 1D np.arrays
        One entry per fold
        Each entry is a 1-dim numpy array of unique integers between 0 to N

    Guarantees for Return Values
    ----------------------------
    Across all folds, guarantee that no two folds put same object in test set.
    For each fold f, we need to guarantee:
    * The *union* of train_ids_per_fold[f] and test_ids_per_fold[f]
    is equal to [0, 1, ... N-1]
    * The *intersection* of the two is the empty set
    * The total size of train and test ids for any fold is equal to N
    '''


test_random_state = np.random.RandomState(seed=42)

# TODO obtain a shuffled order of the n_examples
test_indeces = np.arange(0, test_n_examples+1, 1)
test_random_state.shuffle(test_indeces)    

# TODO establish the row ids that belong to each fold's
# train subset and test subset

test_test_ids_per_fold = np.array_split(test_indeces, test_n_folds)
# test_train_ids_per_fold = np.concatenate([np.delete(test_test_ids_per_fold, i) for i in range(test_n_folds)])



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [10]:
test_test_ids_per_fold

[array([ 5,  0,  9, 10]), array([2, 1, 8, 4]), array([7, 3, 6])]

In [12]:
np.delete(test_test_ids_per_fold, 1, 0)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [26]:
[np.concatenate(list(
    filter(
        lambda current_train_fold: not np.array_equal(current_train_fold, current_test_fold), 
        test_test_ids_per_fold
    ))
) for current_test_fold in test_test_ids_per_fold]

[array([2, 1, 8, 4, 7, 3, 6]),
 array([ 5,  0,  9, 10,  7,  3,  6]),
 array([ 5,  0,  9, 10,  2,  1,  8,  4])]

In [23]:
list(filter(
        lambda current_train_fold: not np.array_equal(current_train_fold, np.array([5, 0, 9, 10])), 
        test_test_ids_per_fold
))

[array([2, 1, 8, 4]), array([7, 3, 6])]